In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
# !pip install ace_tools_open
import ace_tools_open as tools 

In [2]:
# Read csv into df
df = pd.read_csv('../AAPL_technical_analysis.csv')
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)
df

,open,high,low,close,volume,High-Low,High-Close,Low-Close,True Range,ATR,...,-DI,DX,ADX,OBV,RSI,EMA_12,EMA_26,MACD,Signal_Line,MACD_Histogram
Date,,,,,,,,,,,,,,,,,,,,,
1999-11-01,80.00,80.69,77.3700,77.62,2487300,3.3200,NaN,NaN,3.3200,3.320000,...,NaN,NaN,NaN,-2487300,50.000000,77.620000,77.620000,0.000000,0.000000,0.000000
1999-11-02,78.00,81.69,77.3100,80.25,3564600,4.3800,4.07,0.3100,4.3800,3.395714,...,0.000000,100.000000,100.000000,1077300,100.000000,78.024615,77.814815,0.209801,0.041960,0.167840
1999-11-03,81.62,83.25,81.0000,81.50,2932700,2.2500,3.00,0.7500,3.0000,3.367449,...,0.000000,100.000000,100.000000,4010000,100.000000,78.559290,78.087791,0.471498,0.127868,0.343631
1999-11-04,82.06,85.37,80.6200,83.62,3384700,4.7500,3.87,0.8800,4.7500,3.466203,...,0.000000,100.000000,100.000000,7394700,100.000000,79.337861,78.497585,0.840276,0.270349,0.569927
1999-11-05,84.62,88.37,84.0000,88.31,3721500,4.3700,4.75,0.3800,4.7500,3.557902,...,0.000000,100.000000,100.000000,11116200,100.000000,80.718190,79.224430,1.493760,0.515031,0.978728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-14,241.25,245.55,240.9900,244.60,40896227,4.5600,4.02,0.5400,4.5600,6.089485,...,17.202857,30.536939,18.418021,-2518912928,64.955357,235.511931,235.465421,0.046510,-1.669417,1.715927
2025-02-18,244.15,245.18,241.8400,244.47,48822491,3.3400,0.58,2.7600,3.3400,5.893093,...,16.506430,30.536939,19.283658,-2567735419,57.571324,236.890096,236.132427,0.757669,-1.184000,1.941669
2025-02-19,244.66,246.01,243.1604,244.87,32204215,2.8496,1.54,1.3096,2.8496,5.675701,...,15.914473,32.085139,20.198050,-2535531204,56.834532,238.117773,236.779655,1.338119,-0.679576,2.017695


In [3]:
# shifting upward (negative shift) so that feature of time t is associated with target of time t+1
df['close_shifted'] = df['close'].shift(-1)

df = df.dropna(subset=['close_shifted']) # last row will have a NaN close price
df

,open,high,low,close,volume,High-Low,High-Close,Low-Close,True Range,ATR,...,DX,ADX,OBV,RSI,EMA_12,EMA_26,MACD,Signal_Line,MACD_Histogram,close_shifted
Date,,,,,,,,,,,,,,,,,,,,,
1999-11-01,80.00,80.6900,77.3700,77.62,2487300,3.3200,NaN,NaN,3.3200,3.320000,...,NaN,NaN,-2487300,50.000000,77.620000,77.620000,0.000000,0.000000,0.000000,80.25
1999-11-02,78.00,81.6900,77.3100,80.25,3564600,4.3800,4.0700,0.3100,4.3800,3.395714,...,100.000000,100.000000,1077300,100.000000,78.024615,77.814815,0.209801,0.041960,0.167840,81.50
1999-11-03,81.62,83.2500,81.0000,81.50,2932700,2.2500,3.0000,0.7500,3.0000,3.367449,...,100.000000,100.000000,4010000,100.000000,78.559290,78.087791,0.471498,0.127868,0.343631,83.62
1999-11-04,82.06,85.3700,80.6200,83.62,3384700,4.7500,3.8700,0.8800,4.7500,3.466203,...,100.000000,100.000000,7394700,100.000000,79.337861,78.497585,0.840276,0.270349,0.569927,88.31
1999-11-05,84.62,88.3700,84.0000,88.31,3721500,4.3700,4.7500,0.3800,4.7500,3.557902,...,100.000000,100.000000,11116200,100.000000,80.718190,79.224430,1.493760,0.515031,0.978728,96.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-13,236.91,242.3399,235.5700,241.53,53614054,6.7699,5.4699,1.3000,6.7699,6.207137,...,24.821822,17.485797,-2559809155,67.592419,233.859555,234.734655,-0.875100,-2.098399,1.223299,244.60
2025-02-14,241.25,245.5500,240.9900,244.60,40896227,4.5600,4.0200,0.5400,4.5600,6.089485,...,30.536939,18.418021,-2518912928,64.955357,235.511931,235.465421,0.046510,-1.669417,1.715927,244.47
2025-02-18,244.15,245.1800,241.8400,244.47,48822491,3.3400,0.5800,2.7600,3.3400,5.893093,...,30.536939,19.283658,-2567735419,57.571324,236.890096,236.132427,0.757669,-1.184000,1.941669,244.87


In [4]:
# Remove the last n days to test for next n day prediction
n = 10 # days
last_n_rows_df = df.tail(n)

# Store the remaining DataFrame without the last n rows
df = df.iloc[:-n]

In [5]:
# Dataframe after removing the last n day data
df

,open,high,low,close,volume,High-Low,High-Close,Low-Close,True Range,ATR,...,DX,ADX,OBV,RSI,EMA_12,EMA_26,MACD,Signal_Line,MACD_Histogram,close_shifted
Date,,,,,,,,,,,,,,,,,,,,,
1999-11-01,80.000,80.69,77.37,77.62,2487300,3.32,NaN,NaN,3.32,3.320000,...,NaN,NaN,-2487300,50.000000,77.620000,77.620000,0.000000,0.000000,0.000000,80.25
1999-11-02,78.000,81.69,77.31,80.25,3564600,4.38,4.07,0.31,4.38,3.395714,...,100.000000,100.000000,1077300,100.000000,78.024615,77.814815,0.209801,0.041960,0.167840,81.50
1999-11-03,81.620,83.25,81.00,81.50,2932700,2.25,3.00,0.75,3.00,3.367449,...,100.000000,100.000000,4010000,100.000000,78.559290,78.087791,0.471498,0.127868,0.343631,83.62
1999-11-04,82.060,85.37,80.62,83.62,3384700,4.75,3.87,0.88,4.75,3.466203,...,100.000000,100.000000,7394700,100.000000,79.337861,78.497585,0.840276,0.270349,0.569927,88.31
1999-11-05,84.620,88.37,84.00,88.31,3721500,4.37,4.75,0.38,4.75,3.557902,...,100.000000,100.000000,11116200,100.000000,80.718190,79.224430,1.493760,0.515031,0.978728,96.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-01-30,238.665,240.79,237.21,237.59,55658279,3.58,1.43,2.15,3.58,5.725059,...,6.554292,26.381289,-2566982516,45.203679,233.698268,236.409592,-2.711324,-3.454461,0.743136,236.00
2025-01-31,247.190,247.19,233.44,236.00,101075128,13.75,9.60,4.15,13.75,6.298269,...,20.021478,25.927017,-2668057644,49.132830,234.052381,236.379252,-2.326872,-3.228943,0.902071,228.01
2025-02-03,229.990,231.83,225.70,228.01,73063301,6.13,4.17,10.30,10.30,6.584107,...,1.053740,24.150354,-2741120945,44.142988,233.122784,235.759308,-2.636524,-3.110459,0.473935,232.80


In [6]:
# Features (!'close') and target ('close')
X = df.drop(columns=['close', 'close_shifted'])
y = df['close_shifted']

# split data into train/test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 24 features 
print(X_train.shape, y_train.shape)

(5084, 24) (5084,)


In [7]:
# Random Forest Regressor
# params:
# - n_estimators: number of decision trees in the random forest
# - random_state: random seed
rf_model = RandomForestRegressor(n_estimators=100, random_state=42) 

# Fit model
rf_model.fit(X_train, y_train)

# Pred
y_pred = rf_model.predict(X_test)

# Model evaluation 
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Tabulate and compare the actual value with model predicted close price 
results_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
tools.display_dataframe_to_user(name="Random Forest Regression Predictions", dataframe=results_df)

Random Forest Regression Predictions


In [8]:
# Note: MSE Error quantification and Coefficient of Determination
# 1. mse (lower the better)
# 2. r2 score (best r2 score attainable: 1.0, generally bad r2: <0)
mse, r2 

(294.56197799624476, 0.9864259730485289)

In [9]:
display(last_n_rows_df)

,open,high,low,close,volume,High-Low,High-Close,Low-Close,True Range,ATR,...,DX,ADX,OBV,RSI,EMA_12,EMA_26,MACD,Signal_Line,MACD_Histogram,close_shifted
Date,,,,,,,,,,,,,,,,,,,,,
2025-02-06,231.285,233.8000,230.4250,233.22,29925349,3.3750,1.3300,2.0450,3.3750,6.212231,...,6.425153,20.281598,-2705793284,55.498891,233.017208,235.157671,-2.140464,-2.709596,0.569133,227.63
2025-02-07,232.600,234.0000,227.2600,227.63,39707224,6.7400,0.7800,5.9600,6.7400,6.249929,...,1.658792,18.951397,-2745500508,47.600572,232.188406,234.600066,-2.411660,-2.650009,0.238349,227.65
2025-02-10,229.570,230.5850,227.2000,227.65,33115645,3.3850,2.9550,0.4300,3.3850,6.045291,...,1.811059,17.727087,-2712384863,56.014406,231.490190,234.085246,-2.595056,-2.639018,0.043962,232.62
2025-02-11,228.200,235.2300,228.1300,232.62,53718362,7.1000,7.5800,0.4800,7.5800,6.154913,...,9.828981,17.162937,-2658666501,59.674224,231.664007,233.976709,-2.312703,-2.573755,0.261053,236.87
2025-02-12,231.200,236.9600,230.6800,236.87,45243292,6.2800,4.3400,1.9400,6.2800,6.163848,...,13.782640,16.921487,-2613423209,63.340739,232.464929,234.191027,-1.726098,-2.404224,0.678126,241.53
2025-02-13,236.910,242.3399,235.5700,241.53,53614054,6.7699,5.4699,1.3000,6.7699,6.207137,...,24.821822,17.485797,-2559809155,67.592419,233.859555,234.734655,-0.875100,-2.098399,1.223299,244.60
2025-02-14,241.250,245.5500,240.9900,244.60,40896227,4.5600,4.0200,0.5400,4.5600,6.089485,...,30.536939,18.418021,-2518912928,64.955357,235.511931,235.465421,0.046510,-1.669417,1.715927,244.47
2025-02-18,244.150,245.1800,241.8400,244.47,48822491,3.3400,0.5800,2.7600,3.3400,5.893093,...,30.536939,19.283658,-2567735419,57.571324,236.890096,236.132427,0.757669,-1.184000,1.941669,244.87
2025-02-19,244.660,246.0100,243.1604,244.87,32204215,2.8496,1.5400,1.3096,2.8496,5.675701,...,32.085139,20.198050,-2535531204,56.834532,238.117773,236.779655,1.338119,-0.679576,2.017695,245.83


In [10]:
# Test next n days
for d in [2,3,5,10]:
    temp_df = last_n_rows_df.head(d)
    X_next_n_test = temp_df.drop(columns=['close', 'close_shifted'])
    y_next_n_test = temp_df['close_shifted']
    
    y_pred_next_n_test = rf_model.predict(X_next_n_test)

    # Model evaluation 
    mse_n_days = mean_squared_error(y_next_n_test, y_pred_next_n_test)
    r2_n_days = r2_score(y_next_n_test, y_pred_next_n_test)

    print(f'Next {d} days: "mse" = {mse_n_days} "r2" = {r2_n_days}')
    
    

Next 2 days: "mse" = 28.042501839999993 "r2" = -280424.01839971304
Next 3 days: "mse" = 19.64298288000005 "r2" = -2.564150324789223
Next 5 days: "mse" = 28.60831937800055 "r2" = 0.01353071999589861
Next 10 days: "mse" = 16.36244263200023 "r2" = 0.6692908448713206


## Make your next day prediction!

In [11]:
# y_pred = rf_model.predict(example_next_day_data_feature_vector)
# print(f'Prediction: {y_pred}')